In [16]:
import enum
from collections import defaultdict

import spacy
from tqdm import tqdm

In [17]:
nlp = spacy.load("pl_core_news_md")

In [18]:
with open("/home/jdworzans/projects/chatbots/data/task2_questions_with_answers.tsv", "rt") as f:
    content = f.read()

In [20]:
questions_answers = []
for l in content.splitlines():
    question, *answers = l.split("\t")
    questions_answers.append((question, answers))

In [21]:
class QuestionType(enum.Enum):
    UNKNOWN = 0
    YES_NO = 1
    SELECT = 2
    NAME = 3

In [22]:
def determine_type(q: str):
    doc: spacy.tokens.Doc = nlp(q)
    last_sentence: spacy.tokens.Span = list(doc.sents)[-1]
    czy_count = 0
    for token in last_sentence:
        if token.lemma_ == "czy":
            czy_count += 1
    if czy_count > 0:
        if czy_count == 1 and last_sentence[0].lemma_ == "czy":
            return QuestionType.YES_NO
        else:
            return QuestionType.SELECT
    if q.startswith("Jak nazywa"):
        return QuestionType.NAME
    return QuestionType.UNKNOWN

In [23]:
qa_groups = defaultdict(list)

In [24]:
for q, answers in tqdm(questions_answers):
    group = determine_type(q)
    qa_groups[group].append((q, answers))

100%|██████████| 3500/3500 [00:35<00:00, 99.38it/s] 


In [26]:
for group, qa_group in qa_groups.items():
    print(group, len(qa_group))

QuestionType.NAME 616
QuestionType.UNKNOWN 2361
QuestionType.YES_NO 292
QuestionType.SELECT 231


# checks

In [27]:
for q, answers in qa_groups[QuestionType.UNKNOWN]:
    if any(answer.lower() in q.lower() for answer in answers):
        print(q, answers)

W zdaniu: „Szybko przeskoczył przez płot” który wyraz jest przysłówkiem? ['szybko']
Ile medali zdobyli Polacy na Mistrzostwach Świata w lekkiej atletyce w 2013 roku w Moskwie? ['trzy', '3']
Wskaż przyimek w zdaniu „Idę do szkoły”. ['do']
Które miasto jest stolicą Meksyku? ['Meksyk']
Ku czci której greckiej bogini organizowane były panatenaje? ['Atena']
Miłosz, Żeromski, Reymont – który z nich nie dostał Nagrody Nobla? ['Żeromski']
Który rozbiór Polski nastąpił w 1772 roku? ['pierwszy', '1']
Jaki pseudonim nosił Stanisław Wiechecki? ['Wiech']
Ile razy w ciągu życia kwitną agawy? ['raz', '1']
W filmie „Dyktator” Charlie Chaplin zagrał podwójną rolę. A kto był scenarzystą i reżyserem tego filmu? ['Charlie Chaplin']
A2, A3, A4 – który z tych arkuszy papieru ma najmniejsze wymiary? ['A4']
Którą wielką literą alfabetu oznacza się stałą grawitacji Newtona? ['G']
Jak w szachach nazywa się ruch powodujący bezpośrednie zagrożenie króla? ['szach']
Etna, Stromboli, Tambora – który wulkan nie wznos

In [33]:
determine_type("Czy bulaj i iluminator są tym samym?")

<QuestionType.UNKNOWN: 0>

In [29]:
for q, answers in qa_groups[QuestionType.UNKNOWN]:
    if any(answer.lower() in {"tak", "nie"} for answer in answers):
        print(q, answers)

Czy bulaj i iluminator są tym samym? ['nie']
1246 to liczba parzysta? ['tak']


In [13]:
for q, answers in qa_groups[QuestionType.YES_NO]:
    if all(answer.lower() not in {"tak", "nie"} for answer in answers):
        print(q, answers)

Czy choroba Heinego-Medina i dziecięce porażenie to ta sama choroba? ['wokół głowy']


In [14]:
for q, answers in qa_groups[QuestionType.SELECT]:
    if all(answer.lower() not in q.lower() for answer in answers):
        print(q, answers)

Wyspy Zielonego Przylądka leżą bliżej Afryki czy Azji? ['Kto śpiewa piosenkę z czołówki M jak miłość?']
Co ile lat odradzał się z popiołów mityczny feniks – co 50, co 500 czy co 5000 lat? ['co 500 lat']
Miesiąc miodowy to okres przed czy po ślubie? ['przed ślubem']
Czy trybun ludowy był w starożytnym Rzymie urzędnikiem broniącym plebsu czy ministrem spraw zagranicznych? ['urzędnikiem broniącym spraw plebsu']
Kiedy powstał w Nowym Jorku wieżowiec Chryslera: przed I wojną światową czy po niej? ['po I wojnie światowej']
Która liczba jest większa: milion milionów czy tysiąc miliardów? ['są równe']
Wołyń leży na Litwie, Ukrainie czy Białorusi? ['na Ukrainie']
Który liczba jest większa: pierwiastek kwadratowy z 9 czy pierwiastek sześcienny z 27? ['są równe']
Który stopień jest wyższy: pułkownik czy komandor? ['są równe']
Premiera „Dziadów” w reżyserii Wyspiańskiego miała miejsce w XIX czy XX wieku? ['w XX', '20']
Film „Zemsta Sithów” opowiada o Rycerzach Okrągłego Stołu czy rycerzach Jedi? [

In [15]:
for q, answers in qa_groups[QuestionType.UNKNOWN]:
    if q.startswith("Któr") and any(answer.lower() in q.lower() for answer in answers):
        print(q, answers)

Które miasto jest stolicą Meksyku? ['Meksyk']
Który rozbiór Polski nastąpił w 1772 roku? ['pierwszy', '1']
Którą wielką literą alfabetu oznacza się stałą grawitacji Newtona? ['G']
Który wyraz w zdaniu „ten dom jest wysoki” jest przymiotnikiem? ['wysoki']
Którą literę w alfabecie Morse'a oznaczają trzy kropki? ['S']


# Experiments

In [16]:
print("\n".join(map(str, qa_groups[QuestionType.UNKNOWN])))

('W którym państwie rozpoczyna się akcja powieści „W pustyni i w puszczy”?', ['w Egipcie'])
('W jakim zespole występowała Hanka w filmie „Żona dla Australijczyka”?', ['Mazowsze'])
('W którym państwie leży Bombaj?', ['w Indiach'])
('Który numer boczny nosi czołg Rudy z „Czterech pancernych”?', ['102'])
('Co budował w Egipcie inżynier Tarkowski, ojciec Stasia?', ['Kanał Sueski'])
('Kwartet – to ilu wykonawców?', ['czterech', 'czworo', '4'])
('Którzy rzemieślnicy występują w ostatniej powieści Witkacego?', ['szewcy'])
('Hymnem którego kraju jest pieśń „Gwiaździsty sztandar”?', ['Stanów Zjednoczonych', 'USA'])
('Jak brzmi ulubione powiedzonko klucznika Horeszków Gerwazego?', ['mopanku'])
('Którego gazu jest najwięcej w powietrzu?', ['azotu'])
('Kto jest autorem obrazu namalowanego w 1892 r. „Kobiety z Tahiti”?', ['Paul Gauguin'])
('Jak ma na imię brat Fidela Castro, który przejął po nim władzę?', ['Raul'])
('Co znaczy pochodzące z francuskiego słowo aliant?', ['sojusznik'])
('Jak z greckie

In [22]:
from spacy import displacy

In [43]:
doc = nlp("Hajducy w wojsku polskim w XVI i XVII wieku walczyli pieszo czy konno?")

In [44]:
doc.ents

(Hajducy, polskim, XVI, wieku)

In [42]:
displacy.render(doc, "ent")

In [27]:
for q, answers in questions_answers:
    doc = nlp(q)
    if len(list(doc.sents)) > 1:
        print(q)

W którym państwie znajdowała się tzw. Bizonia – połączone strefy okupacyjne Wlk. Brytanii i USA?
Na autoportrecie z Saskią Rembrandt przedstawił siebie z kobietą siedzącą na kolanach. Kim była ta kobieta dla Rembrandta?
Kto wcześniej był koronowany na króla Polski: Jagiełło czy Jadwiga?
Kora z zespołem Maanam śpiewa: „Chmury wiszą nad miastem”. Nad którym?
Rzeźbiarz Sławomir Dunikowski swój dorobek przekazał państwu. Gdzie możemy podziwiać jego muzeum?
„Chrząszcz brzmi w trzcinie” – kto jest autorem tych słów?
W którym państwie wznosi się najwyższy szczyt Ameryki Płd. – Aconcagua?
Jakiej narodowości był pisarz Kallimach, nauczyciel synów Kazimierza Jagiellończyka?
W starej piosence śpiewano „kupujcie bubliczki”. Czy są to małe bukieciki kwiatów?
Jakiego koloru były wzgórza Afryki w tytule powieści Ernesta Hemingwaya?
Śpiewamy: „Poszła Karolinka do Gogolina”. Z jakiego regionu wywodzi się Karolina?
Według Ignacego Krasickiego „miłe złego początki, lecz koniec...”?
Jakiego koloru jest po

In [20]:
idx = 2
q = qa_groups[QuestionType.UNKNOWN][idx][0]
q

'W którym państwie leży Bombaj?'

In [23]:
doc = nlp(q)
displacy.render(doc)

In [55]:
sentence = list(doc.sents)[0]
sentence.root

rozpoczyna

In [84]:
sentence[sentence.root.n_lefts:]

państwie rozpoczyna się akcja powieści „W pustyni i w puszczy”?

In [81]:
sentence.root.right_edge

?

In [78]:
list(sentence.root.rights)

[się, akcja, ?]

In [63]:
list(sentence.root.lefts)

[którym, państwie]

nazywa

In [26]:
doc.get_lca_matrix

<function Doc.get_lca_matrix>

In [6]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

Wapno nsubj:pass palone ADJ []
palone nsubj związek NOUN [Wapno]
to cop związek NOUN []
związek ROOT związek NOUN [palone, to, chemiczny, pierwiastek, ?]
chemiczny amod związek NOUN []
czy cc pierwiastek NOUN []
pierwiastek conj związek NOUN [czy]
? punct związek NOUN []


In [7]:
for q in [
    "Sahara Zachodnia posiada duże złoża miedzi, fosforytów czy uranu?",
    "Wapno palone to związek chemiczny czy pierwiastek?",
]:
    doc = nlp(q)
    displacy.render(doc, style="dep")

In [8]:
displacy.render(doc, style="dep")

In [10]:
list(doc)

[Wapno, palone, to, związek, chemiczny, czy, pierwiastek, ?]

In [11]:
import re

In [13]:
from typing import Dict, Tuple

import pandas as pd
import requests

SOLR_URL = "http://localhost:8983/solr/dialogs/query"


def query_solr(query: str, language: str) -> Tuple[str, Dict]:
    """
    Function to query the Solr database with a question

    Parameters
    ----------
        query, str:
            The question to be searched
        language, str:
            Question language abbreviation
    """
    if query is None:
        return None, None

    solr_query = f"content_txt_{language}: {query} OR "
    solr_query += " OR ".join([f"content_txt_{language}:{t}" for t in query.split()])

    r = requests.get(
        SOLR_URL,
        json={"query": solr_query, "params": {"debugQuery": True}}
    )

    if not r.ok:
        return None, r.text
    response = r.json()
    docs = response["response"]["docs"]
    if not docs:
        return None, response
    info = {
        "solr_query": solr_query,
        "response": response,
    }
    return docs[0][f"title_txt_{language}"], info

In [14]:
Q = "Wapno palone to związek chemiczny czy pierwiastek?"

In [15]:
for description, *opts in re.findall("(.*?) to (.*?) czy (.*?)\?", Q):
    counts = []
    for opt in opts:
        q = description + " to " + opt
        solr_query = f'content_txt_pl:"{description}" AND content_txt_pl:"{opt}"'
        r = requests.get(
            SOLR_URL,
            json={"query": solr_query, "params": {"debugQuery": False}}
        )
        response = r.json()
        docs = response["response"]["docs"]
        counts.append((len(docs), opt))

In [16]:
sorted(counts)

[(0, 'pierwiastek'), (5, 'związek chemiczny')]

In [18]:
from tqdm import tqdm

In [27]:
a.split("\n")[0]

'Jak nazywa się pierwsza litera alfabetu greckiego?\talfa'

In [50]:
re.findall("^(.*?) to (.*?) czy (.*?)\?\s*(.*?)$", 'Wapno palone to związek chemiczny czy pierwiastek?\tzwiązek chemiczny')

[('Wapno palone', 'związek chemiczny', 'pierwiastek', 'związek chemiczny')]

In [105]:
with open("data/questions.tsv", "rt") as f:
    a = f.read()

In [72]:
re.findall("^(.*?) to (.*?)(?:, (.*?))* czy (.*?)\?\s*(.*?)$", "Oryks to antylopa, pies czy kot?	antylopa")

[('Oryks', 'antylopa', 'pies', 'kot', 'antylopa')]

In [70]:
solr_query

'content_txt_pl:punkt OR content_txt_pl:przecięcia OR content_txt_pl:osi OR content_txt_pl:w OR content_txt_pl:układzie OR content_txt_pl:współrzędnych AND '

In [100]:
re.findall("(.*?) to (.*?)(?:, (.*?))* czy (.*?)\?", "Ktoś „nie w ciemię bity” to ktoś sprytny czy naiwny?")

[('Ktoś „nie w ciemię bity”', 'ktoś sprytny', '', 'naiwny')]

In [85]:
re.search("\w*", "abc")[0]

'abc'

In [91]:
def tokenize(word):
    return filter(None, [
        re.search("\w*", t)[0]
        for t in word.lower().split()
    ])

In [98]:
response

{'responseHeader': {'status': 0,
  'QTime': 0,
  'params': {'json': '{"query": "content_txt_pl:diablotka AND content_txt_pl:ostro OR content_txt_pl:przyprawiona OR content_txt_pl:zupa OR content_txt_pl:rybna", "params": {"debugQuery": false}}'}},
 'response': {'numFound': 0, 'start': 0, 'numFoundExact': True, 'docs': []}}

In [107]:
for l in re.finditer("^(.*?) to (.*?)(?:, (.*?))* czy (.*?)\?\s*(.*?)$", a, flags=re.MULTILINE):
    print(l[0])

Punkt przecięcia osi w układzie współrzędnych to punkt zero czy linia odcięcia?	punkt zero
Baron to tytuł wyższy czy niższy niż hrabia?	niższy
Ktoś „nie w ciemię bity” to ktoś sprytny czy naiwny?	sprytny
Czy sombrero to kapelusz, danie czy taniec?	kapelusz
Marmury Elgina to świątynia w Atenach czy zbiór starożytnych rzeźb greckich?	zbiór starożytnych rzeźb greckich
Wapno palone to związek chemiczny czy pierwiastek?	związek chemiczny
Średniowieczne gęśle to instrument szarpany czy dęty?	szarpany
Miesiąc miodowy to okres przed czy po ślubie?	przed ślubem
Hakownica to broń sieczna czy palna?	palna	Londyn
Świekier to dawna polska nazwa ojca żony czy ojca męża?	ojca męża
Dybuk to dusza zmarłego grzesznika wstępująca do ciała żywego czy duch w domostwie domowym?	dusza zmarłego grzesznika wstępująca do ciała żywego
Hakownica to sprzęt alpejski czy broń?	broń
Diablotka to ostro przyprawiona zupa rybna czy placek z ciasta francuskiego?	placek
Synogarlice to odmiana gołębi czy sikorek?	gołębi
Ju

In [104]:
a

<re.Match object; span=(241130, 241194), match='Urwistki to komórki glonów czy odnóża świerszczy?>

In [99]:
for description, *opts, correct in re.findall("^(.*?) to (.*?)(?:, (.*?))* czy (.*?)\?\s*(.*?)$", a, flags=re.MULTILINE):
    counts = []
    description_query = " OR ".join([f"content_txt_pl:{t}" for t in tokenize(description)])
    for opt in filter(None, opts):
        opt_query = " OR ".join([f"content_txt_pl:{t}" for t in tokenize(opt)])
        solr_query = f"{description_query} AND {opt_query}"
        r = requests.get(
            SOLR_URL,
            json={"query": solr_query}
        )
        response = r.json()
        counts.append((response["response"]["numFound"], opt))
    print(description + " to " + correct, list(reversed(sorted(counts))))

Punkt przecięcia osi w układzie współrzędnych to punkt zero [(247, 'punkt zero'), (66, 'linia odcięcia')]
Baron to niższy [(495, 'tytuł wyższy'), (29, 'niższy niż hrabia')]
Ktoś „nie w ciemię bity” to sprytny [(1931, 'ktoś sprytny'), (1, 'naiwny')]
Czy sombrero to kapelusz [(3, 'kapelusz'), (1, 'taniec'), (1, 'danie')]
Marmury Elgina to zbiór starożytnych rzeźb greckich [(1, 'świątynia w Atenach'), (0, 'zbiór starożytnych rzeźb greckich')]
Wapno palone to związek chemiczny [(67, 'związek chemiczny'), (3, 'pierwiastek')]
Średniowieczne gęśle to szarpany [(10, 'instrument szarpany'), (0, 'dęty')]
Miesiąc miodowy to przed ślubem [(240, 'po ślubie'), (23, 'okres przed')]
Hakownica to palna	Londyn [(10, 'broń sieczna'), (8, 'palna')]
Świekier to ojca męża [(0, 'ojca męża'), (0, 'dawna polska nazwa ojca żony')]
Dybuk to dusza zmarłego grzesznika wstępująca do ciała żywego [(2, 'dusza zmarłego grzesznika wstępująca do ciała żywego'), (1, 'duch w domostwie domowym')]
Hakownica to broń [(10, 'b